# Making an env

In [ ]:
import gymnasium as gym
from matplotlib import pyplot as plt
%matplotlib inline

env = gym.make('highway-v0', render_mode='rgb_array')
env.reset()
for _ in range(3):
    action = env.action_type.actions_indexes["IDLE"]
    obs, reward, done, truncated, info = env.step(action)
    env.render()

plt.imshow(env.render())
plt.show()
env.close()

# Configuring an env

In [ ]:
import pprint

env = gym.make("highway-v0", render_mode='rgb_array')
pprint.pprint(env.config)

In [ ]:
env.config["lanes_count"] = 2
env.reset()
plt.imshow(env.render())
plt.show()
env.close()

# Training

In [ ]:
import gymnasium as gym
import highway_env
from stable_baselines3 import DQN

env = gym.make("highway-fast-v0")
model = DQN('MlpPolicy', env,
              policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=5e-4,
              buffer_size=15000,
              learning_starts=200,
              batch_size=32,
              gamma=0.8,
              train_freq=1,
              gradient_steps=1,
              target_update_interval=50,
              verbose=1,
              tensorboard_log="highway_dqn/")
model.learn(int(2e4))
model.save("highway_dqn/model")

# Load and test saved model
model = DQN.load("highway_dqn/model")
while True:
  done = truncated = False
  obs, info = env.reset()
  while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    env.render()

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import DQN

import highway_env
#highway_env.register_highway_envs()

env = gym.make("highway-fast-v0", render_mode="rgb_array")
# Run the trained model and record video
model = DQN.load("highway_dqn/model", env=env)
env = RecordVideo(env, video_folder="highway_dqn/videos", episode_trigger=lambda e: True)
env.unwrapped.set_record_video_wrapper(env)
#env.configure({"simulation_frequency": 15})  # Higher FPS for rendering

for videos in range(10):
    done = truncated = False
    obs, info = env.reset()
    while not (done or truncated):
        # Predict
        action, _states = model.predict(obs, deterministic=True)
        # Get reward
        obs, reward, done, truncated, info = env.step(action)
        # Render
        env.render()
env.close()


In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import DQN, DDPG, PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import SubprocVecEnv

import highway_env


TRAIN = False

if __name__ == '__main__':
    n_cpu = 6
    batch_size = 64
    env = make_vec_env("racetrack-v0", n_envs=n_cpu, vec_env_cls=SubprocVecEnv)
    model = PPO("MlpPolicy",
                env,
                policy_kwargs=dict(net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
                n_steps=batch_size * 12 // n_cpu,
                batch_size=batch_size,
                n_epochs=10,
                learning_rate=5e-4,
                gamma=0.9,
                verbose=2,
                tensorboard_log="racetrack_ppo/")
    # Train the model
    if TRAIN:
        model.learn(total_timesteps=int(1e5))
        model.save("racetrack_ppo/model")
        del model

    # Run the algorithm
    model = PPO.load("racetrack_ppo/model", env=env)

    env = gym.make("racetrack-v0", render_mode="rgb_array")
    env = RecordVideo(env, video_folder="racetrack_ppo/videos", episode_trigger=lambda e: True)
    env.unwrapped.set_record_video_wrapper(env)

    for video in range(10):
        done = truncated = False
        obs, info = env.reset()
        while not (done or truncated):
            # Predict
            action, _states = model.predict(obs, deterministic=True)
            # Get reward
            obs, reward, done, truncated, info = env.step(action)
            # Render
            env.render()
    env.close()


In [3]:
import numpy as np
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import SAC, DDPG, PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import SubprocVecEnv

import highway_env

env = gym.make('parking-v0')

N_EPISODES = 10  # @param {type: "integer"}

model = SAC.load("parking_her/model", env=env)

env = gym.make('parking-v0', render_mode='rgb_array')
env = RecordVideo(env, video_folder="parking_her/videos", episode_trigger=lambda e: True)
for episode in range(N_EPISODES):
    obs, info = env.reset()
    done = truncated = False
    while not (done or truncated):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


d:\HDX\.conda\envs\highway2\lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-0.mp4.
Moviepy - Writing video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-0.mp4
Moviepy - Building video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-1.mp4.
Moviepy - Writing video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-1.mp4
Moviepy - Building video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-2.mp4.
Moviepy - Writing video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-2.mp4



...

Moviepy - Done !
Moviepy - video ready d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-6.mp4
Moviepy - Building video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-7.mp4.
Moviepy - Writing video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-7.mp4



Moviepy - Done !
Moviepy - video ready d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-7.mp4
Moviepy - Building video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-8.mp4.
Moviepy - Writing video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-8.mp4



Moviepy - Done !
Moviepy - video ready d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-8.mp4


Moviepy - Building video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-9.mp4.
Moviepy - Writing video d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-9.mp4



Moviepy - Done !
Moviepy - video ready d:\HDX\SJTU\大三\AI\2023-SJTU-AI-HW\Project1\code\mycode\parking_her\videos\rl-video-episode-9.mp4
